# Лабораторна робота: Основні парадигми ООП у Python

## 1. Інкапсуляція

In [1]:
import random

class BankAccount:
    def __init__(self, owner, balance):
        self.owner = owner
        self.__balance = balance  # Приватний атрибут

    def deposit(self, amount):
        self.__balance += amount
        return f"Поповнено на {amount}"

    def withdraw(self, amount):
        if amount <= self.__balance:
            self.__balance -= amount
            return f"Знято {amount}"
        return "Недостатньо коштів"

    def get_balance(self):
        return self.__balance

account = BankAccount("Bohdan", 1000)

print(f"Початковий баланс: {account.get_balance()}")
for i in range(5):
    val = random.randint(50, 300)
    action = random.choice([account.deposit, account.withdraw])
    print(f"Операція {i+1}: {action(val)}, Баланс: {account.get_balance()}")

Початковий баланс: 1000
Операція 1: Поповнено на 73, Баланс: 1073
Операція 2: Знято 220, Баланс: 853
Операція 3: Поповнено на 114, Баланс: 967
Операція 4: Знято 237, Баланс: 730
Операція 5: Знято 280, Баланс: 450


## 2. Наслідування

In [2]:
class Vehicle:
    def __init__(self, brand, model):
        self.brand = brand
        self.model = model

    def display_info(self):
        return f"{self.brand} {self.model}"
    
    def service_info(self): # ⭐️ Новий метод
        return f"Транспортний засіб {self.brand} потребує технічного огляду."

class Car(Vehicle):
    def __init__(self, brand, model, seats):
        super().__init__(brand, model)
        self.seats = seats

car = Car("Toyota", "Camry", 5)
print(car.display_info())
print(car.service_info()) # Виклик успадкованого методу

Toyota Camry
Транспортний засіб Toyota потребує технічного огляду.


## 3. Поліморфізм

In [3]:
class Animal:
    def speak(self):
        return "Тварина не видає звуків за замовчуванням."

class Dog(Animal):
    def speak(self):
        return "Woof!"

class Fish(Animal): # ⭐️ Клас без метода speak
    pass

animals = [Dog(), Fish()]
for a in animals:
    print(f"{type(a).__name__}: {a.speak()}")
# Пояснення: Fish використовує метод speak базового класу Animal.

Dog: Woof!
Fish: Тварина не видає звуків за замовчуванням.


## 4. Абстракція та фінальна гра 

In [5]:
from abc import ABC, abstractmethod
from random import randint, choice

# 1. Абстракція: створюємо базовий клас для всіх типів зброї
class Item(ABC):
    def __init__(self, name: str, health=500):
        self.name = name
        self.health = health
    
    @abstractmethod
    def attack(self, another_item):
        pass

    @abstractmethod
    def boost(self):
        pass

# 2. Наслідування та Інкапсуляція: Меч
class Sword(Item):
    def __init__(self, name, attack_power: int):
        super().__init__(name=name)
        self.__attack_power = attack_power  # Приватний атрибут
        self._sharp = 0
    
    def attack(self, another_item: Item):
        damage = self.__attack_power + self._sharp + randint(0, 10)
        another_item.health -= damage
        return f"⚔️ {self.name} наносить {damage} шкоди мечем."
    
    def boost(self):
        self._sharp += 7
        return f"✨ {self.name} нагострив меч (+7 до атаки)."

# 3. Наслідування: Сокира
class Axe(Item):
    def __init__(self, name, attack_power: int):
        super().__init__(name=name)
        self.__attack_power = attack_power
    
    def attack(self, another_item: Item):
        damage = self.__attack_power + randint(0, 25)
        another_item.health -= damage
        return f"🪓 {self.name} рубає сокирою на {damage} шкоди."

    def boost(self):
        self.health += 40
        return f"🛡️ {self.name} використав щит (+40 до HP)."

# 4. Поліморфізм: Лук (Індивідуальне завдання)
class Bow(Item):
    def __init__(self, name, attack_power: int, range_power: int):
        super().__init__(name=name)
        self.__attack_power = attack_power
        self.range_power = range_power
    
    def attack(self, another_item: Item):
        damage = self.__attack_power + randint(5, 15) + self.range_power
        another_item.health -= damage
        return f"🏹 {self.name} стріляє з лука на {damage} шкоди."

    def boost(self):
        self.range_power += 3
        return f"🎯 {self.name} покращив приціл (+3 до дальності)."

# --- Автоматична симуляція бою для відображення на GitHub ---

# Випадковий вибір персонажів
weapons = [
    Sword("Ескалібур", 35),
    Axe("Бойова сокира", 30),
    Bow("Довгий лук", 25, 10)
]

player = choice(weapons)
enemy = choice([w for w in weapons if w != player])

print(f"🔥 ПОЧАТОК БОЮ: {player.name} проти {enemy.name}! 🔥")
print("-" * 50)

round_num = 1
while player.health > 0 and enemy.health > 0:
    print(f"\nРАУНД {round_num}")
    print(f"{player.name}: {player.health} HP | {enemy.name}: {enemy.health} HP")
    
    # Автоматичний вибір дії: 80% шанс атаки, 20% шанс підсилення
    player_action = "1" if randint(1, 100) <= 80 else "2"
    
    if player_action == "1":
        print(player.attack(enemy))
    else:
        print(player.boost())
        
    # Хід ворога (якщо він вижив)
    if enemy.health > 0:
        enemy_action = "1" if randint(1, 100) <= 80 else "2"
        if enemy_action == "1":
            print(enemy.attack(player))
        else:
            print(enemy.boost())
    
    round_num += 1
    if round_num > 50: break # Запобіжник від вічного циклу

print("-" * 50)
if player.health > 0:
    print(f"🏆 ПЕРЕМОГА: {player.name} виграв бій!")
else:
    print(f"💀 ПОРАЗКА: {enemy.name} здолав гравця.")

🔥 ПОЧАТОК БОЮ: Довгий лук проти Ескалібур! 🔥
--------------------------------------------------

РАУНД 1
Довгий лук: 500 HP | Ескалібур: 500 HP
🏹 Довгий лук стріляє з лука на 50 шкоди.
⚔️ Ескалібур наносить 43 шкоди мечем.

РАУНД 2
Довгий лук: 457 HP | Ескалібур: 450 HP
🏹 Довгий лук стріляє з лука на 45 шкоди.
⚔️ Ескалібур наносить 43 шкоди мечем.

РАУНД 3
Довгий лук: 414 HP | Ескалібур: 405 HP
🏹 Довгий лук стріляє з лука на 45 шкоди.
⚔️ Ескалібур наносить 38 шкоди мечем.

РАУНД 4
Довгий лук: 376 HP | Ескалібур: 360 HP
🏹 Довгий лук стріляє з лука на 47 шкоди.
⚔️ Ескалібур наносить 38 шкоди мечем.

РАУНД 5
Довгий лук: 338 HP | Ескалібур: 313 HP
🏹 Довгий лук стріляє з лука на 46 шкоди.
⚔️ Ескалібур наносить 40 шкоди мечем.

РАУНД 6
Довгий лук: 298 HP | Ескалібур: 267 HP
🏹 Довгий лук стріляє з лука на 42 шкоди.
⚔️ Ескалібур наносить 43 шкоди мечем.

РАУНД 7
Довгий лук: 255 HP | Ескалібур: 225 HP
🏹 Довгий лук стріляє з лука на 47 шкоди.
⚔️ Ескалібур наносить 43 шкоди мечем.

РАУНД 8
Довгий